In [27]:
import tensorflow as tf
import numpy as np
from __future__ import absolute_import, division, print_function


In [28]:
from tensorflow.keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [29]:
num_features = 784
x_train, x_test = np.array(x_train, np.float32), np.array(x_test, np.float32)
x_train, x_test = x_train.reshape([-1, num_features]), x_test.reshape([-1, num_features])
x_train, x_test = x_train / 255., x_test / 255.  
print(x_train.shape)
print(x_test.shape)
print(x_train[0])

(60000, 784)
(10000, 784)
[0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.     

In [30]:
num_classes = 10 # as there are 0-9 digits
num_features = 784
learning_rate = 0.01
training_steps = 1000
batch_size = 256
display_step = 50

In [31]:
train_data=tf.data.Dataset.from_tensor_slices((x_train,y_train))
train_data=train_data.repeat().shuffle(5000).batch(batch_size).prefetch(1)

In [32]:
# train_data

In [33]:
W = tf.Variable(tf.ones([num_features, num_classes]), name="weight")
b = tf.Variable(tf.zeros([num_classes]), name="bias")

In [34]:
def logistic_regression(x):
    return tf.nn.softmax(tf.matmul(x, W) + b)
def cross_entropy(y_pred, y_true):
    y_true = tf.one_hot(y_true, depth=num_classes)
    y_pred = tf.clip_by_value(y_pred, 1e-9, 1.)
    return tf.reduce_mean(-tf.reduce_sum(y_true * tf.math.log(y_pred)))

In [35]:
def accuracy(y_pred, y_true):
  c = tf.equal(tf.argmax(y_pred, 1), tf.cast(y_true, tf.int64))
  return tf.reduce_mean(tf.cast(c, tf.float32))


In [36]:
optimizer = tf.optimizers.SGD(learning_rate)
def run_optimization(x, y):
    with tf.GradientTape() as g:
        pred = logistic_regression(x)
        loss = cross_entropy(pred, y)
    gradients = g.gradient(loss, [W, b])
    optimizer.apply_gradients(zip(gradients, [W, b]))

In [37]:
for step, (batch_x, batch_y) in enumerate(train_data.take(training_steps), 1):
    run_optimization(batch_x, batch_y)
    if step % display_step == 0:
        pred = logistic_regression(batch_x)
        loss = cross_entropy(pred, batch_y)
        acc = accuracy(pred, batch_y)
        print("step: %i, loss: %f, accuracy: %f" % (step, loss, acc))

step: 50, loss: 102.577866, accuracy: 0.878906
step: 100, loss: 222.953430, accuracy: 0.832031
step: 150, loss: 151.797821, accuracy: 0.875000
step: 200, loss: 199.614014, accuracy: 0.835938
step: 250, loss: 87.433128, accuracy: 0.890625
step: 300, loss: 458.124817, accuracy: 0.632812
step: 350, loss: 48.810268, accuracy: 0.929688
step: 400, loss: 49.501442, accuracy: 0.941406
step: 450, loss: 54.434856, accuracy: 0.949219
step: 500, loss: 73.631546, accuracy: 0.941406
step: 550, loss: 59.911777, accuracy: 0.933594
step: 600, loss: 98.313141, accuracy: 0.914062
step: 650, loss: 69.859390, accuracy: 0.914062
step: 700, loss: 43.626602, accuracy: 0.949219
step: 750, loss: 71.650772, accuracy: 0.914062
step: 800, loss: 52.036339, accuracy: 0.925781
step: 850, loss: 49.262493, accuracy: 0.941406
step: 900, loss: 109.722015, accuracy: 0.898438
step: 950, loss: 73.882195, accuracy: 0.921875
step: 1000, loss: 33.852184, accuracy: 0.945312


In [38]:
pred = logistic_regression(x_test)
print("Test Accuracy: %f" % accuracy(pred, y_test))

Test Accuracy: 0.908800


In [39]:
# re running gives dirrerent accuracy , maybe becase of splitting randomly